In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

# **Reading the dataset**

In [ ]:
data_path = "/content/drive/MyDrive/001 My Skills/002 CS Engineering   Automated Math (BPHC)/004 Data Science (DS)   Artificial Intelligence (AI)/005 Textual Data (Unstructured Data) (Sequential Data)/004 NLP Tasks/2. NLG Tasks/0. Machine Translation (MT)/1. MonoLingual Machine Translation (MT)/2. NMT/french_english_dataset.txt"
# you can download other sentence pair dataset from here - http://www.manythings.org/anki/
# or use WMT dataset - http://www.statmt.org/wmt16/

In [ ]:
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

lines

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)',
 'Go.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)',
 'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)',
 'Hi.\tSalut.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)',
 'Run!\tCours\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)',
 'Run!\tCourez\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)',
 'Run!\tPrenez vos jambes à vos cous !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077449 (sacredceltic)',
 'Run!\tFile !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077454 (sacredceltic)',
 'Run!\tFilez !\tCC-BY 2.0 (France) Attribution: tatoeba

# **Data Preprocessing**


In [ ]:
num_samples = 10000  # Number of samples to train on. # train-test split basically
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [ ]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [ ]:
print(input_texts)
print("Number of samples:", len(input_texts))
max_encoder_seq_length = max([len(txt) for txt in input_texts])
print("Max sequence length for inputs:", max_encoder_seq_length)

['Go.', 'Go.', 'Go.', 'Hi.', 'Hi.', 'Run!', 'Run!', 'Run!', 'Run!', 'Run!', 'Run!', 'Run!', 'Run!', 'Run.', 'Run.', 'Run.', 'Run.', 'Run.', 'Run.', 'Run.', 'Run.', 'Who?', 'Wow!', 'Duck!', 'Duck!', 'Duck!', 'Fire!', 'Help!', 'Hide.', 'Hide.', 'Jump!', 'Jump.', 'Stop!', 'Stop!', 'Stop!', 'Wait!', 'Wait!', 'Wait!', 'Wait.', 'Wait.', 'Wait.', 'Wait.', 'Begin.', 'Begin.', 'Go on.', 'Go on.', 'Go on.', 'Hello!', 'Hello!', 'I see.', 'I see.', 'I try.', 'I won!', 'I won!', 'I won.', 'Oh no!', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Smile.', 'Smile.', 'Smile.', 'Sorry?', 'Attack!', 'Attack!', 'Attack!', 'Buy it.', 'Buy it.', 'Buy it.', 'Buy it.', 'Cheers!', 'Cheers!', 'Cheers!', 'Cheers!', 'Eat it.', 'Eat it.', 'Get up.', 'Get up.', 'Get up.', 'Go now.', 'Go now.', 'Go now.', 'Got it!', 'Got it!', 'Got it!', 'Got it?', 'Got it?', 'Got it?', 'Hop in.', 'Hop in.', 'Hug me.', 'Hug me.', 'I fell.', 'I fell.', 'I fled.

In [ ]:
print(target_texts)
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print("Max sequence length for outputs:", max_decoder_seq_length)

['\tVa !\n', '\tMarche.\n', '\tBouge !\n', '\tSalut !\n', '\tSalut.\n', '\tCours\u202f!\n', '\tCourez\u202f!\n', '\tPrenez vos jambes à vos cous !\n', '\tFile !\n', '\tFilez !\n', '\tCours !\n', '\tFuyez !\n', '\tFuyons !\n', '\tCours\u202f!\n', '\tCourez\u202f!\n', '\tPrenez vos jambes à vos cous !\n', '\tFile !\n', '\tFilez !\n', '\tCours !\n', '\tFuyez !\n', '\tFuyons !\n', '\tQui ?\n', '\tÇa alors\u202f!\n', '\tÀ terre\xa0!\n', '\tBaisse-toi\xa0!\n', '\tBaissez-vous\xa0!\n', '\tAu feu !\n', "\tÀ l'aide\u202f!\n", '\tCache-toi.\n', '\tCachez-vous.\n', '\tSaute.\n', '\tSaute.\n', '\tÇa suffit\u202f!\n', '\tStop\u202f!\n', '\tArrête-toi !\n', '\tAttends !\n', '\tAttendez !\n', '\tAttendez.\n', '\tAttends !\n', '\tAttendez !\n', '\tAttends.\n', '\tAttendez.\n', '\tCommencez.\n', '\tCommence.\n', '\tPoursuis.\n', '\tContinuez.\n', '\tPoursuivez.\n', '\tBonjour !\n', '\tSalut !\n', '\tJe comprends.\n', '\tAha.\n', "\tJ'essaye.\n", "\tJ'ai gagné !\n", "\tJe l'ai emporté !\n", '\tJ’ai gagn

In [ ]:
print(input_characters)
input_characters = sorted(list(input_characters))
print(input_characters)
num_encoder_tokens = len(input_characters)
print("Number of unique input tokens:", num_encoder_tokens)
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
print(input_token_index) # this represents the index which will have 1 in OHE of these characters and rest index will have zeroes

{'V', ',', 'U', 'G', '!', 'l', ' ', '8', '3', 's', 'T', 'D', 'a', '-', 'i', 'E', 'm', 'O', 'J', 'I', '0', '%', "'", '?', '9', 'K', 'o', 'H', '5', 'B', 't', 'é', 'n', 'e', 'j', 'v', 'W', ':', 'x', 'p', 'N', 'w', 'R', 'A', 'P', '$', '"', 'L', 'g', '.', 'Q', 'z', 'c', 'S', 'r', 'q', 'M', 'k', '&', 'Y', '2', 'y', '7', 'b', 'd', 'h', 'F', '1', 'u', 'C', 'f'}
[' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '5', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é']
Number of unique input tokens: 71
{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, '&': 5, "'": 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '5': 14, '7': 15, '8': 16, '9': 17, ':': 18, '?': 19, 'A': 20, 'B': 21, 'C': 22, 'D': 23, 'E': 24, 'F': 25, 'G': 26, 'H': 27, 'I': 28, 'J

In [ ]:
print(target_characters)
target_characters = sorted(list(target_characters))
print(target_characters)
num_decoder_tokens = len(target_characters)
print("Number of unique output tokens:", num_decoder_tokens)
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print(target_token_index) # this represents the index which will have 1 in OHE of these characters and rest index will have zeroes

{'8', '-', 'D', 'ô', 'J', 'o', '\xa0', 'à', 'é', 'n', 'e', 'p', 'R', 'Q', '’', 'z', 'c', 'q', 'd', 'F', '1', 'ù', 'Ê', '»', 'V', 'l', '!', '3', 's', 'T', 'a', 'ç', 'i', 'î', '\n', '%', "'", 'û', 'Ç', '\u202f', '5', 'v', ':', '.', 'r', 'b', '\t', 'ï', '«', '2', 'œ', 'm', 'I', '0', 'â', 'K', 'H', 'ê', 't', '\u2009', 'j', 'x', 'L', '(', 'S', '&', 'M', 'É', 'u', 'C', ',', 'U', 'G', ' ', 'E', 'O', 'è', '?', '9', 'B', ')', 'N', 'w', 'A', 'P', 'g', 'k', 'Y', 'y', 'h', 'À', 'f'}
['\t', '\n', ' ', '!', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '5', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']
Number of unique output tokens: 92
{'\t': 0, '\n

In [ ]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
print(encoder_input_data)
print(encoder_input_data.shape)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
(10000, 15

In [ ]:
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
print(decoder_input_data)
print(decoder_input_data.shape)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
(10000, 59

In [ ]:
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
print(decoder_target_data)
print(decoder_target_data.shape)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
(10000, 59

In [ ]:
# adding 1s in respective positions to create OHE vectors

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.0

  encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0

  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.0
    if t > 0:
      # decoder_target_data will be ahead by one timestep and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

  decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
  decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
print(encoder_input_data[0][0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
print(decoder_input_data[0][0])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
print(decoder_target_data[0][0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# **Modelling, Training and Prediction**

In [ ]:
import tensorflow as tf
from tensorflow import keras

### Defining the model architecture


In [ ]:
latent_dim = 256  # Timestamps

Single hidden layer LSTM encoder network 


In [ ]:
# input layer of encoder
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))

# hidden layer of encoder
encoder = keras.layers.LSTM(latent_dim, return_state=True)

# getting the outputs - here state_h represent hidden state and state_c represents cell states
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` because as we know in joint training we dont require these outputs at every time stamp of encoder and require only the hidden state and the cell state
encoder_states = [state_h, state_c]

Single hidden layer LSTM decoder network


In [ ]:
# input layer of decoder
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# hidden layer of decoder (using `encoder_states` as initial state)
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True) #We don't use the return states in the training model, but we will use them in inference.
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Output layer of decoder - returns full output sequences and to return internal states as well. 
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax") # using the optional V wala layer and optional softmax because they are necessary here (refer time series with RNN notes)
decoder_outputs = decoder_dense(decoder_outputs)

Defining the entire model

In [ ]:
# Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

### Training the model


In [ ]:
# this took around 1hr to train
model.fit (
  [encoder_input_data, decoder_input_data],
  decoder_target_data,
  batch_size=64,            # Batch size for training.
  epochs=100,               # Number of epochs to train for.
  validation_split=0.2,
)

Epoch 1/100
125/125 [==============================] - 43s 320ms/step - loss: 1.1485 - accuracy: 0.7348 - val_loss: 1.0182 - val_accuracy: 0.7137
Epoch 2/100
125/125 [==============================] - 41s 328ms/step - loss: 0.8208 - accuracy: 0.7782 - val_loss: 0.8264 - val_accuracy: 0.7691
Epoch 3/100
125/125 [==============================] - 41s 328ms/step - loss: 0.6572 - accuracy: 0.8155 - val_loss: 0.6983 - val_accuracy: 0.7997
Epoch 4/100
125/125 [==============================] - 42s 338ms/step - loss: 0.5714 - accuracy: 0.8343 - val_loss: 0.6423 - val_accuracy: 0.8130
Epoch 5/100
125/125 [==============================] - 42s 338ms/step - loss: 0.5231 - accuracy: 0.8465 - val_loss: 0.5917 - val_accuracy: 0.8308
Epoch 6/100
125/125 [==============================] - 41s 326ms/step - loss: 0.4881 - accuracy: 0.8563 - val_loss: 0.5566 - val_accuracy: 0.8376
Epoch 7/100
125/125 [==============================] - 42s 337ms/step - loss: 0.4612 - accuracy: 0.8636 - val_loss: 0.5375 -

In [ ]:
# Save model
model.save("s2s")

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


### Inference (sampling)



In [ ]:
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2

In [ ]:
# Inference Step
'''
In inference mode, when we want to decode a test sentence, we:
- Encode the input sequence into state vectors
- Start with a target sequence of size 1 (just the start-of-sequence character)
- Feed the state vectors and 1-char target sequence to the decoder to produce predictions for the next character
- Sample the next character using these predictions (we simply use argmax).
- Append the sampled character to the target sequence
- Repeat until we generate the end-of-sequence character or wehit the character limit.
'''

decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm (decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
  # Encode the input as state vectors.
  states_value = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens))
    
  # Populate the first character of target sequence with the start character.
  target_seq[0, 0, target_token_index["\t"]] = 1.0

  # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = ""
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char

    # Exit condition: either hit max length or find stop character.
    if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
        stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.0

    # Update states
    states_value = [h, c]
    
  return decoded_sentence

In [ ]:
# generating decoded sentences

for seq_index in range(20):
  # Take one sequence (part of the training set) for trying out decoding.
  input_seq = encoder_input_data[seq_index : seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print("-")
  print("Input sentence:", input_texts[seq_index])
  print("Decoded sentence:", decoded_sentence)


-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: File !

